In [32]:
import praw
import pandas as pd
  
reddit_read_only = praw.Reddit(client_id="dOLvr35kk_81cIumKlA8yQ",        
                               client_secret="Z1iMBY4HN_MWshEPU9GoqRGhtTlN9A",      
                               user_agent="Old-Ad9688")        
  
  
subreddit = reddit_read_only.subreddit("redditdev")
  

print("Display Name:", subreddit.display_name)
  

print("Title:", subreddit.title)
  

print("Description:", subreddit.description)

Display Name: redditdev
Title: reddit Development
Description: A subreddit for discussion of Reddit's API and Reddit API clients.

* [Read the API Overview & Rules](https://github.com/reddit/reddit/wiki/API)
* [Check out the API documentation](http://www.reddit.com/dev/api)
* [PRAW chat](https://join.slack.com/t/praw/shared_invite/enQtOTUwMDcxOTQ0NzY5LWVkMGQ3ZDk5YmQ5MDEwYTZmMmJkMTJkNjBkNTY3OTU0Y2E2NGRlY2ZhZTAzMWZmMWRiMTMwYjdjODkxOGYyZjY)
* [Snoowrap chat](https://gitter.im/not-an-aardvark/snoowrap)
* [Unofficial Discord](https://discord.gg/hVMhxpV)
* Please do not request bots here. Consider /r/requestabot instead.


Please confine discussion to Reddit's API instead of using this as a soapbox to talk to the admins. In particular, use [/r/ideasfortheadmins](/r/ideasfortheadmins) for feature ideas and [/r/bugs](/r/bugs) for bugs. If you have general reddit questions, try [/r/help](/r/help).

To see an explanation of recent user-facing changes to reddit (and the code behind them), check o

In [34]:
 
subreddit = reddit_read_only.subreddit("wine")
# Obtenha todos os posts do subreddit
posts = subreddit.new(limit=None)

posts_dict = {
    "Title": [],
    "Post Text": [],
    "ID": [],
    "Score": [],
    "Total Comments": [],
    "Post URL": [],
    "Comments": []  # Lista de comentários associados a cada postagem
}

for post in posts:
    # Título de cada post
    posts_dict["Title"].append(post.title)

    # Texto dentro do post, tratando texto ausente
    if post.selftext:
        posts_dict["Post Text"].append(post.selftext)
    else:
        posts_dict["Post Text"].append("")

    # ID único de cada post
    posts_dict["ID"].append(post.id)

    # A pontuação de um post
    posts_dict["Score"].append(post.score)

    # Total de comentários dentro do post
    posts_dict["Total Comments"].append(post.num_comments)

    # URL de cada post
    posts_dict["Post URL"].append(post.url)

    # Obter os comentários da postagem
    comments = []
    post.comments.replace_more(limit=None)  # Expandir todos os comentários
    for comment in post.comments.list():
        comments.append(comment.body)
    posts_dict["Comments"].append(comments)

# Salvar os dados em um DataFrame do pandas
top_posts = pd.DataFrame(posts_dict)
print(top_posts)
pd.DataFrame(posts_dict)
top_posts

                                                 Title  \
0               How do you use these wine descriptors?   
1                                     Music to my ears   
2                Suggestions on alternatives to Lebraz   
3    Got engaged and celebrated by drinking a bottl...   
4    What feature(s) would you need in order to jus...   
..                                                 ...   
987   What glasses does Memento Mori use for tastings?   
988                                           New Here   
989                                   Quick BTG at MSP   
990  Recommendations on age-worthy reds for a weddi...   
991            Suisun Valley wine tasting suggestions?   

                                             Post Text       ID  Score  \
0    I’ve noticed that wine pros use certain descri...  13qniw5      1   
1                                                       13qnhbl      1   
2    Hi everyone!\n\nThanks for this wonderful comm...  13qn5sb      1   
3      

,Title,Post Text,ID,Score,Total Comments,Post URL,Comments
0,How do you use these wine descriptors?,I’ve noticed that wine pros use certain descri...,13qniw5,1,0,https://www.reddit.com/r/wine/comments/13qniw5...,[]
1,Music to my ears,,13qnhbl,1,1,https://i.redd.it/qijgyb9ngs1b1.jpg,[Thank you for your submission to r/wine! Plea...
2,Suggestions on alternatives to Lebraz,Hi everyone!\n\nThanks for this wonderful comm...,13qn5sb,1,0,https://www.reddit.com/r/wine/comments/13qn5sb...,[]
3,Got engaged and celebrated by drinking a bottl...,,13qkml9,12,6,https://i.imgur.com/iVV9g0v.jpg,[Thank you for your submission to r/wine! Plea...
4,What feature(s) would you need in order to jus...,,13qjay3,2,9,https://www.reddit.com/r/wine/comments/13qjay3...,[I wouldn't pay to be honest.\nThey make money...
...,...,...,...,...,...,...,...
987,What glasses does Memento Mori use for tastings?,I believe the glasses may have WS or SW on the...,12udwie,70,19,https://i.redd.it/ozsutczzpbva1.jpg,[Thank you for your submission to r/wine! Plea...
988,New Here,Hey everyone I'm new to wine. Do most people d...,12ub209,0,30,https://www.reddit.com/r/wine/comments/12ub209...,"[Wife: ""sure, I'll have some wine with you""\n\..."
989,Quick BTG at MSP,,12uamy3,1,6,https://i.redd.it/6xo4h8bt6bva1.jpg,[Thank you for your submission to r/wine! Plea...
990,Recommendations on age-worthy reds for a weddi...,"Hi there,\nMy fiance and I are getting married...",12u88u2,5,16,https://www.reddit.com/r/wine/comments/12u88u2...,[Opening a bottle of wine and taking sips thro...


In [35]:

  
top_posts.to_excel("Top Posts.xlsx", index=True)